# GWExPy TimeSeries 相互運用性 (Interop) チュートリアル

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tatsuki-washimi/gwexpy/blob/main/docs/web/en/user_guide/tutorials/intro_interop.ipynb)

このノートブックでは、`gwexpy` の `TimeSeries` クラスに追加された新しい相互運用機能 (Interoperability Features) を紹介します。
`gwexpy` は、Pandas, Xarray, PyTorch, Astropy などの一般的なデータサイエンス・ライブラリとの間で、シームレスにデータを変換することができます。

## 目次

- 1. 汎用データ形式・データ基盤
    - 1.1. NumPy [Original GWpy]
    - 1.2. Pandas との連携 [Original GWpy]
    - 1.3. Polars [GWExPy New]
    - 1.4. Xarray との連携 [Original GWpy]
    - 1.5. Dask との連携 [GWExPy New]
    - 1.6. JSON / Python Dict (to_json) (to_json) [GWExPy New]
- 2. データベース・ストレージ層
    - 2.1. HDF5 [Original GWpy]
    - 2.2. SQLite [GWExPy New]
    - 2.3. Zarr [GWExPy New]
    - 2.4. netCDF4 [GWExPy New]
- 3. 情報科学・機械学習・加速計算
    - 3.1. PyTorch との連携 [GWExPy New]
    - 3.2. CuPy (CUDA 加速) [GWExPy New]
    - 3.3. TensorFlow との連携 [GWExPy New]
    - 3.4. JAX との連携 [GWExPy New]
- 4. 天文学・重力波物理学
    - 4.1. PyCBC / LAL [Original GWpy]
    - 4.2. Astropy との連携 [Original GWpy]
    - 4.3. Specutils との連携 [GWExPy New]
    - 4.4. Pyspeckit との連携 [GWExPy New]
- 5. 素粒子物理学・高エネルギー物理
    - 5.1. CERN ROOT との連携 [GWExPy New]
    - 5.2. ROOT オブジェクトからの復元 (`from_root`)
- 6. 地球物理学・地震学・電磁気学
    - 6.1. ObsPy [Original GWpy]
    - 6.2. SimPEG との連携 [GWExPy New]
    - 6.3. MTH5 / MTpy [GWExPy New]
- 7. 音響・音声解析
    - 7.1. Librosa / Pydub [GWExPy New]
- 8. 医学・生体信号解析
    - 8.1. MNE-Python [GWExPy New]
    - 8.2. Elephant / quantities との連携 [GWExPy New]
    - 8.3. Neo [GWExPy New]
- 9. まとめ


In [ ]:
import os

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

In [ ]:
import logging
import warnings

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from gwpy.time import LIGOTimeGPS

from gwexpy.timeseries import TimeSeries

warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
warnings.filterwarnings("ignore", category=UserWarning)

# Suppress MTH5 redundant warnings
logging.getLogger("mth5.groups.base").setLevel(logging.ERROR)
logging.getLogger("mth5").setLevel(logging.ERROR)

In [ ]:
# Create sample data
# 10seconds 100Hz Sine waveData Generate 
rate = 100 * u.Hz
dt = 1 / rate
t0 = LIGOTimeGPS(1234567890, 0)
duration = 10 * u.s
size = int(rate * duration)
times = np.arange(size) * dt.value
data = np.sin(2 * np.pi * 1.0 * times) # 1Hz sine wave

ts = TimeSeries(data, t0=t0, dt=dt, unit="V", name="demo_signal")
print("Original TimeSeries:")
print(ts)
ts.plot(title="Original TimeSeries");

## 1. 汎用データ形式・データ基盤


### 1.1. NumPy <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **NumPy**: NumPy は Python の数値計算の基盤となるライブラリで、多次元配列と高速な数学演算を提供します。
> 📚 [NumPy Documentation](https://numpy.org/)

`TimeSeries.value` や `np.asarray(ts)` で NumPy 配列を取得できます。


### 1.2. Pandas との連携 <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **Pandas**: Pandas はデータ分析・操作のための強力なライブラリで、DataFrame と Series による柔軟なデータ構造を提供します。
> 📚 [Pandas Documentation](https://pandas.pydata.org/)

`to_pandas()` メソッドを使うと、`TimeSeries` を `pandas.Series` に変換できます。
インデックスは `datetime` (UTC), `gps`, `seconds` (Unix timestamp) から選択可能です。


In [ ]:
try:
 # Pandas Series conversion ( datetime index)
 s_pd = ts.to_pandas(index="datetime")
 print("\n--- Converted to Pandas Series ---")
 display(s_pd)
 s_pd.plot(title="Pandas Series")
 plt.show()
 plt.close()

 # Pandas Series from TimeSeries Restoration
 ts_restored = TimeSeries.from_pandas(s_pd, unit="V")
 print("\n--- Restored TimeSeries from Pandas ---")
 print(ts_restored)

 del s_pd, ts_restored

except ImportError:
 print("Pandas is not installed.")

### 1.3. Polars <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **Polars**: Polars は Rust で実装された高速な DataFrame ライブラリで、大規模データの高速処理に優れています。
> 📚 [Polars Documentation](https://pola.rs/)

`to_polars()` で Polars DataFrame/Series に変換できます。


In [ ]:
try:
 import polars as pl

 _ = pl
 # TimeSeries -> Polars DataFrame
 df_pl = ts.to_polars()
 print("--- Polars DataFrame ---")
 print(df_pl.head())

 # Plot using Polars/Matplotlib
 plt.figure()
 data_col = [c for c in df_pl.columns if c != "time"][0]
 plt.plot(df_pl["time"], df_pl[data_col])
 plt.title("Polars Data Plot")
 plt.show()

 # Recover to TimeSeries
 from gwexpy.timeseries import TimeSeries

 ts_recovered = TimeSeries.from_polars(df_pl)
 print("Recovered from Polars:", ts_recovered)

 del df_pl, ts_recovered

except ImportError:
 print("Polars not installed.")

### 1.4. Xarray との連携 <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **xarray**: xarray は多次元ラベル付き配列のためのライブラリで、NetCDF データの操作や気象・地球科学データの解析に広く使用されます。
> 📚 [xarray Documentation](https://docs.xarray.dev/)

`xarray` は多次元のラベル付き配列を扱う強力なライブラリです。`to_xarray()` でメタデータを保持したまま変換できます。


In [ ]:
try:
 # Xarray DataArray conversion
 da = ts.to_xarray()
 print("\n--- Converted to Xarray DataArray ---")
 print(da)
 # Data (attrs) Verify
 print("Attributes:", da.attrs)

 da.plot()
 plt.title("Xarray DataArray")
 plt.show()
 plt.close()

 # Restoration
 ts_x = TimeSeries.from_xarray(da)
 print("\n--- Restored TimeSeries from Xarray ---")
 print(ts_x)

 del da, ts_x

except ImportError:
 print("Xarray is not installed.")

### 1.5. Dask との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **Dask**: Dask は並列計算のためのライブラリで、NumPy/Pandas を超える大規模データの処理を可能にします。
> 📚 [Dask Documentation](https://www.dask.org/)


In [ ]:
try:
 import dask.array as da

 _ = da
 # Dask Array conversion
 dask_arr = ts.to_dask(chunks="auto")
 print("\n--- Converted to Dask Array ---")
 print(dask_arr)

 # (compute=True )
 ts_from_dask = TimeSeries.from_dask(dask_arr, t0=ts.t0, dt=ts.dt, unit=ts.unit)
 print("Recovered from Dask:", ts_from_dask)

 del dask_arr, ts_from_dask

except ImportError:
 print("Dask not installed.")

### 1.6. JSON / Python Dict (to_json) <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **JSON**: JSON (JavaScript Object Notation) は軽量なデータ交換フォーマットで、Python 標準ライブラリでサポートされています。
> 📚 [JSON Documentation](https://docs.python.org/3/library/json.html)

`to_json()` や `to_dict()` で JSON 互換の辞書形式を出力できます。


In [ ]:
import json

# TimeSeries -> JSON string
ts_json = ts.to_json()
print("--- JSON Representation (Partial) ---")
print(ts_json[:500] + "...")

# Plot data by loading back from JSON
ts_dict_temp = json.loads(ts_json)
plt.figure()
plt.plot(ts_dict_temp["data"])
plt.title("Plot from JSON Data")
plt.show()

# Recover from JSON
from gwexpy.timeseries import TimeSeries

ts_recovered = TimeSeries.from_json(ts_json)
print("Recovered from JSON:", ts_recovered)

del ts_json, ts_dict_temp, ts_recovered

## 2. データベース・ストレージ層


### 2.1. HDF5 <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **HDF5**: HDF5 は大規模な科学データを効率的に保存・管理するための階層型データフォーマットです。h5py ライブラリで Python から利用できます。
> 📚 [HDF5 Documentation](https://www.h5py.org/)

`to_hdf5_dataset()` で HDF5 への保存をサポート。


In [ ]:
try:
 import tempfile

 import h5py

 with tempfile.NamedTemporaryFile(suffix=".h5") as tmp:
 # TimeSeries -> HDF5
 with h5py.File(tmp.name, "w") as f:
 ts.to_hdf5_dataset(f, "dataset_01")

 # Read back and display
 with h5py.File(tmp.name, "r") as f:
 ds = f["dataset_01"]
 print("--- HDF5 Dataset Info ---")
 print(f"Shape: {ds.shape}, Dtype: {ds.dtype}")
 print("Attributes:", dict(ds.attrs))

 # Recover
 from gwexpy.timeseries import TimeSeries

 ts_recovered = TimeSeries.from_hdf5_dataset(f, "dataset_01")
 print("Recovered from HDF5:", ts_recovered)

 ts_recovered.plot()
 plt.title("Recovered from HDF5")
 plt.show()

 del ds, ts_recovered

except ImportError:
 print("h5py not installed.")

### 2.2. SQLite <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **SQLite**: SQLite は軽量な組み込み型 SQL データベースエンジンで、Python 標準ライブラリに含まれています。
> 📚 [SQLite Documentation](https://docs.python.org/3/library/sqlite3.html)

`to_sqlite()` で DB 永続化をサポート。


In [ ]:
import sqlite3

conn = sqlite3.connect(":memory:")

# TimeSeries -> SQLite
series_id = ts.to_sqlite(conn, series_id="test_series")
print(f"Saved to SQLite with ID: {series_id}")

# Verify data in SQL
cursor = conn.cursor()
row = cursor.execute("SELECT * FROM series WHERE series_id=?", (series_id,)).fetchone()
print("Metadata from SQL:", row)

# Recover
from gwexpy.timeseries import TimeSeries

ts_recovered = TimeSeries.from_sqlite(conn, series_id)
print("Recovered from SQLite:", ts_recovered)

ts_recovered.plot()
plt.title("Recovered from SQLite")
plt.show()

del series_id, conn, cursor, ts_recovered

### 2.3. Zarr <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **Zarr**: Zarr はチャンク化・圧縮された多次元配列のためのストレージフォーマットで、クラウドストレージとの連携に優れています。
> 📚 [Zarr Documentation](https://zarr.dev/)

`to_zarr()` でクラウドストレージ向き形式に対応。


In [ ]:
try:
 import os
 import tempfile

 import zarr

 with tempfile.TemporaryDirectory() as tmpdir:
 store_path = os.path.join(tmpdir, "test.zarr")
 # TimeSeries -> Zarr
 ts.to_zarr(store_path, path="timeseries")

 # Read back
 z = zarr.open(store_path, mode="r")
 ds = z["timeseries"]
 print("--- Zarr Array Info ---")
 print(ds.info)

 # Recover
 from gwexpy.timeseries import TimeSeries

 ts_recovered = TimeSeries.from_zarr(store_path, "timeseries")
 print("Recovered from Zarr:", ts_recovered)

 ts_recovered.plot()
 plt.title("Recovered from Zarr")
 plt.show()

 del z, ds, ts_recovered

except ImportError:
 print("zarr not installed.")

### 2.4. netCDF4 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **netCDF4**: netCDF4 は気象・海洋・地球科学データの標準フォーマットで、自己記述的なデータ構造を持ちます。
> 📚 [netCDF4 Documentation](https://unidata.github.io/netcdf4-python/)

`to_netcdf4()` で気象・海洋データの標準に対応。


In [ ]:
try:
 import tempfile
 from pathlib import Path

 import netCDF4

 with tempfile.TemporaryDirectory() as tmpdir:
 path = Path(tmpdir) / "gwexpy_timeseries.nc"

 # TimeSeries -> netCDF4
 with netCDF4.Dataset(path, "w", format="NETCDF3_CLASSIC") as ds:
 ts.to_netcdf4(ds, "my_signal")

 # Read back
 with netCDF4.Dataset(path, "r") as ds:
 v = ds.variables["my_signal"]
 print("--- netCDF4 Variable Info ---")
 print(v)

 # Recover
 from gwexpy.timeseries import TimeSeries

 ts_recovered = TimeSeries.from_netcdf4(ds, "my_signal")
 print("Recovered from netCDF4:", ts_recovered)

 ts_recovered.plot()
 plt.title("Recovered from netCDF4")
 plt.show()

 del v, ts_recovered

except ImportError:
 print("netCDF4 not installed.")


## 3. 情報科学・機械学習・加速計算


### 3.1. PyTorch との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **PyTorch**: PyTorch は深層学習フレームワークで、動的計算グラフと GPU 加速をサポートします。
> 📚 [PyTorch Documentation](https://pytorch.org/)

ディープラーニングの前処理として、`TimeSeries` を直接 `torch.Tensor` に変換できます。GPU転送も可能です。


In [ ]:
import os

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")

try:
 import torch

 # PyTorch Tensor conversion
 tensor = ts.to_torch(dtype=torch.float32)
 print("\n--- Converted to PyTorch Tensor ---")
 print(f"Tensor shape: {tensor.shape}, dtype: {tensor.dtype}")

 # Tensor from Restoration (t0, dt )
 ts_torch = TimeSeries.from_torch(tensor, t0=ts.t0, dt=ts.dt, unit="V")
 print("\n--- Restored from Torch ---")
 print(ts_torch)

 del tensor, ts_torch

except ImportError:
 print("PyTorch is not installed.")

### 3.2. CuPy (CUDA 加速) <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **CuPy**: CuPy は NumPy 互換の GPU 配列ライブラリで、NVIDIA CUDA を使用した高速計算を可能にします。
> 📚 [CuPy Documentation](https://cupy.dev/)

GPU 上での計算を可能にする CuPy 配列へ変換できます。


In [ ]:
from gwexpy.interop import is_cupy_available

if is_cupy_available():
 import cupy as cp

 # TimeSeries -> CuPy
 y_gpu = ts.to_cupy()
 print("--- CuPy Array (on GPU) ---")
 print(y_gpu)

 # Simple processing on GPU
 y_gpu_filt = y_gpu * 2.0

 # Plot (must move to CPU for plotting)
 plt.figure()
 plt.plot(cp.asnumpy(y_gpu_filt))
 plt.title("CuPy Data (Moved to CPU for plot)")
 plt.show()

 # Recover
 from gwexpy.timeseries import TimeSeries

 ts_recovered = TimeSeries.from_cupy(y_gpu_filt, t0=ts.t0, dt=ts.dt)
 print("Recovered from CuPy:", ts_recovered)

 del y_gpu, y_gpu_filt, ts_recovered

else:
 print("CuPy or CUDA driver not available.")

### 3.3. TensorFlow との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **TensorFlow**: TensorFlow は Google が開発した機械学習プラットフォームで、大規模な本番環境での利用に優れています。
> 📚 [TensorFlow Documentation](https://www.tensorflow.org/)


In [ ]:
try:
 import os
 import warnings

 os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
 warnings.filterwarnings(
 "ignore", category=UserWarning, module=r"google\.protobuf\..*"
 )
 warnings.filterwarnings(
 "ignore", category=UserWarning, message=r"Protobuf gencode version.*"
 )
 import tensorflow as tf

 _ = tf
 # TensorFlow Tensor conversion
 tf_tensor = ts.to_tensorflow()
 print("\n--- Converted to TensorFlow Tensor ---")
 print(f"Tensor shape: {tf_tensor.shape}")
 print(f"Tensor dtype: {tf_tensor.dtype}")

 # ts_from_tensorflow = TimeSeries.from_tensorflow(
 tf_tensor, t0=ts.t0, dt=ts.dt, unit=ts.unit
 )
 print("Recovered from TF:", ts_from_tensorflow)

 del tf_tensor, ts_from_tensorflow

except ImportError:
 pass

### 3.4. JAX との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **JAX**: JAX は Google が開発した高性能数値計算ライブラリで、自動微分と XLA コンパイルによる高速化が特徴です。
> 📚 [JAX Documentation](https://jax.readthedocs.io/)


In [ ]:
try:
 import os

 os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
 import jax

 _ = jax
 import jax.numpy as jnp

 _ = jnp
 # JAX Array conversion
 jax_arr = ts.to_jax()
 print("\n--- Converted to JAX Array ---")
 print(f"Array shape: {jax_arr.shape}")

 # ts_from_jax = TimeSeries.from_jax(jax_arr, t0=ts.t0, dt=ts.dt, unit=ts.unit)
 print("Recovered from JAX:", ts_from_jax)

 del jax_arr, ts_from_jax
except ImportError:
 print("JAX not installed.")

## 4. 天文学・重力波物理学


### 4.1. PyCBC / LAL <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **LAL**: LAL (LIGO Algorithm Library) は LIGO/Virgo の公式解析ライブラリで、重力波解析の基盤を提供します。
> 📚 [LAL Documentation](https://lscsoft.docs.ligo.org/lalsuite/)

> **PyCBC**: PyCBC は重力波データ解析のためのライブラリで、信号探索やパラメータ推定に使用されます。
> 📚 [PyCBC Documentation](https://pycbc.org/)

重力波解析の標準ツールとの互換性を備えています。


### 4.2. Astropy との連携 <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **Astropy**: Astropy は天文学のための Python ライブラリで、座標変換、時間系変換、単位系などをサポートします。
> 📚 [Astropy Documentation](https://www.astropy.org/)

天文学分野で標準的な `astropy.timeseries.TimeSeries` との相互変換もサポートしています。


In [ ]:
try:
 # Astropy TimeSeries conversion
 ap_ts = ts.to_astropy_timeseries()
 print("\n--- Converted to Astropy TimeSeries ---")
 print(ap_ts[:5])
 fig, ax = plt.subplots()
 ax.plot(ap_ts.time.jd, ap_ts["value"])
 plt.title("Astropy TimeSeries")
 plt.show()
 plt.close()

 # Restoration
 ts_astro = TimeSeries.from_astropy_timeseries(ap_ts)
 print("\n--- Restored from Astropy ---")
 print(ts_astro)

 del ap_ts, ts_astro

except ImportError:
 print("Astropy is not installed.")

### 4.3. Specutils との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **specutils**: specutils は天文スペクトルデータの操作・解析のための Astropy 関連パッケージです。
> 📚 [specutils Documentation](https://specutils.readthedocs.io/)

`FrequencySeries` は、Astropy エコシステムのスペクトル解析ライブラリ `specutils` の `Spectrum1D` オブジェクトと相互変換可能です。
単位 (Units) や周波数軸が適切に保持されます。


In [ ]:
try:
 import specutils

 _ = specutils

 from gwexpy.frequencyseries import FrequencySeries

 # FrequencySeries -> specutils.Spectrum1D
 fs = FrequencySeries(
 np.random.random(100), frequencies=np.linspace(10, 100, 100), unit="Jy"
 )
 spec = fs.to_specutils()
 print("specutils Spectrum1D:", spec)
 print("Spectral axis unit:", spec.spectral_axis.unit)

 # specutils.Spectrum1D -> FrequencySeries
 fs_rec = FrequencySeries.from_specutils(spec)
 print("Restored FrequencySeries unit:", fs_rec.unit)

 del fs, spec, fs_rec

except ImportError:
 print("specutils library not found. Skipping example.")

### 4.4. Pyspeckit との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **PySpecKit**: PySpecKit は電波天文学のスペクトル解析ツールキットで、スペクトル線のフィッティングなどをサポートします。
> 📚 [PySpecKit Documentation](https://pyspeckit.readthedocs.io/)

`FrequencySeries` は、汎用的なスペクトル解析ツールキット `pyspeckit` の `Spectrum` オブジェクトとも連携可能です。


In [ ]:
try:
 import pyspeckit

 _ = pyspeckit
 from gwexpy.frequencyseries import FrequencySeries

 # FrequencySeries -> pyspeckit.Spectrum
 fs = FrequencySeries(np.random.random(100), frequencies=np.linspace(10, 100, 100))
 spec = fs.to_pyspeckit()
 print("pyspeckit Spectrum length:", len(spec.data))

 # pyspeckit.Spectrum -> FrequencySeries
 fs_rec = FrequencySeries.from_pyspeckit(spec)
 print("Restored FrequencySeries length:", len(fs_rec))

 del fs, spec, fs_rec

except ImportError:
 print("pyspeckit library not found. Skipping example.")

## 5. 素粒子物理学・高エネルギー物理


### 5.1. CERN ROOT との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **ROOT**: ROOT は CERN が開発した高エネルギー物理学向けのデータ解析フレームワークです。
> 📚 [ROOT Documentation](https://root.cern/)

高エネルギー物理学で標準的なツールである **ROOT** との相互運用が強化されました。
`gwexpy` の Series オブジェクトを ROOT の `TGraph` や `TH1D`, `TH2D` に高速に変換したり、ROOT ファイルを作成することができます。
逆に、ROOT オブジェクトから `TimeSeries` などを復元することも可能です。

**Note**: この機能を使用するには `ROOT` (PyROOT) がインストールされている必要があります。


In [ ]:
try:
 import numpy as np
 import ROOT

 from gwexpy.timeseries import TimeSeries

 # Data preparation
 t = np.linspace(0, 10, 1000)
 data = np.sin(2 * np.pi * 1.0 * t) + np.random.normal(0, 0.5, size=len(t))
 ts = TimeSeries(data, dt=t[1] - t[0], name="signal")

 # --- 1. TGraph conversion ---
 # Vector done conversion
 graph = ts.to_tgraph()

 # ROOT Plot
 c1 = ROOT.TCanvas("c1", "TGraph Example", 800, 600)
 graph.SetTitle("ROOT TGraph;GPS Time [s];Amplitude")
 graph.Draw("AL")
 c1.Draw()
 # c1.SaveAs("signal_graph.png") # Savedo 

 print(f"Created TGraph: {graph.GetName()} with {graph.GetN()} points")

 # --- 2. TH1D ( ) conversion ---
 # conversion ( )
 hist = ts.to_th1d()

 c2 = ROOT.TCanvas("c2", "TH1D Example", 800, 600)
 hist.SetTitle("ROOT TH1D;GPS Time [s];Amplitude")
 hist.SetLineColor(ROOT.kRed)
 hist.Draw()
 c2.Draw()

 print(f"Created TH1D: {hist.GetName()} with {hist.GetNbinsX()} bins")

 del t, data, graph, hist, c1, c2

except ImportError:
 pass
except Exception as e:
 print(f"An error occurred: {e}")

### 5.2. ROOT オブジェクトからの復元 (`from_root`)

> **ROOT**: ROOT は CERN が開発した高エネルギー物理学向けのデータ解析フレームワークです。
> 📚 [ROOT Documentation](https://root.cern/)

既存の ROOT ファイルにあるヒストグラムやグラフを読み込んで、解析しやすい `gwexpy` オブジェクトに戻すことができます。


In [ ]:
try:
 if "hist" in locals() and hist:
 # ROOT TH1D -> TimeSeries
 # Load histogram bin contents as TimeSeries data
 ts_restored = from_root(TimeSeries, hist)

 print(f"Restored TimeSeries: {ts_restored.name}")
 print(ts_restored)

 # TGraph from Restoration 
 ts_from_graph = from_root(TimeSeries, graph)
 print(f"Restored from TGraph: {len(ts_from_graph)} samples")

 del ts_restored, ts_from_graph

except NameError:
 pass # hist graph Create 
except ImportError:
 pass

## 6. 地球物理学・地震学・電磁気学


### 6.1. ObsPy <span style="color: #3498db; font-weight: bold;">[Original GWpy]</span>

> **ObsPy**: ObsPy は地震学データの取得・処理・解析のための Python ライブラリで、MiniSEED 形式などをサポートします。
> 📚 [ObsPy Documentation](https://docs.obspy.org/)

地震学で標準的な ObsPy との相互運用をサポート。


In [ ]:
try:
 import obspy

 _ = obspy
 # TimeSeries -> ObsPy Trace
 tr = ts.to_obspy()
 print("--- ObsPy Trace ---")
 print(tr)

 # Plot using ObsPy
 tr.plot()

 # Recover to TimeSeries
 from gwexpy.timeseries import TimeSeries

 ts_recovered = TimeSeries.from_obspy(tr)
 print("Recovered from ObsPy:", ts_recovered)

 del tr, ts_recovered

except ImportError:
 print("ObsPy not installed.")

In [ ]:
try:
 import obspy

 _ = obspy
 tr = ts.to_obspy()
 print("ObsPy Trace:", tr)

 del tr

except ImportError:
 print("ObsPy not installed.")

### 6.2. SimPEG との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **SimPEG**: SimPEG は地球物理学的逆問題のシミュレーション・推定フレームワークです。
> 📚 [SimPEG Documentation](https://simpeg.xyz/)

`gwexpy` は、地球物理学のフォワード計算・反転モデリングライブラリである `SimPEG` との連携をサポートしています。
`TimeSeries` (TDEM) および `FrequencySeries` (FDEM) を `simpeg.data.Data` オブジェクトに変換したり、その逆を行ったりできます。


In [ ]:
try:
 import numpy as np
 import simpeg

 _ = simpeg
 from simpeg import maps

 _ = maps

 from gwexpy.frequencyseries import FrequencySeries
 from gwexpy.timeseries import TimeSeries

 # --- TimeSeries -> SimPEG (TDEM ) ---
 ts = TimeSeries(np.random.normal(size=100), dt=0.01, unit="A/m^2")
 simpeg_data_td = ts.to_simpeg(location=np.array([0, 0, 0]))
 print("SimPEG TDEM data shape:", simpeg_data_td.dobs.shape)

 # --- FrequencySeries -> SimPEG (FDEM ) ---
 fs = FrequencySeries(
 np.random.normal(size=10) + 1j * 0.1, frequencies=np.logspace(0, 3, 10)
 )
 simpeg_data_fd = fs.to_simpeg(location=np.array([0, 0, 0]), orientation="z")
 print("SimPEG FDEM data shape:", simpeg_data_fd.dobs.shape)

 del simpeg_data_td, simpeg_data_fd, fs

except ImportError:
 print("SimPEG library not found. Skipping example.")

### 6.3. MTH5 / MTpy <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **MTH5**: MTH5 は磁気地電流 (MT) データのための HDF5 ベースのデータフォーマットです。
> 📚 [MTH5 Documentation](https://mth5.readthedocs.io/)

地磁気地電流法データの MTH5 保存に対応。


In [ ]:
try:
 import logging
 import tempfile

 logging.getLogger("mth5").setLevel(logging.ERROR)
 logging.getLogger("mt_metadata").setLevel(logging.ERROR)

 import mth5

 _ = mth5
 with tempfile.NamedTemporaryFile(suffix=".h5") as tmp:
 from gwexpy.interop.mt_ import from_mth5, to_mth5

 # TimeSeries -> MTH5
 # We need to provide station and run names for MTH5 structure
 to_mth5(ts, tmp.name, station="SITE01", run="RUN01")
 print(f"Saved to MTH5 file: {tmp.name}")

 # Display structure info if needed, or just recover
 # Recover
 ts_recovered = from_mth5(tmp.name, "SITE01", "RUN01", ts.name or "Ex")
 print("Recovered from MTH5:", ts_recovered)

 ts_recovered.plot()
 plt.title("Recovered from MTH5")
 plt.show()

 del ts_recovered

except ImportError:
 print("mth5 not installed.")

## 7. 音響・音声解析


### 7.1. Librosa / Pydub <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **pydub**: pydub はオーディオファイルの操作（編集・変換・エフェクト）のためのシンプルなライブラリです。
> 📚 [pydub Documentation](https://github.com/jiaaro/pydub)

> **librosa**: librosa は音声・音楽解析のためのライブラリで、スペクトル解析やビート検出などの機能を提供します。
> 📚 [librosa Documentation](https://librosa.org/)

音声処理ライブラリ Librosa や Pydub との連携をサポート。


In [ ]:
try:
 import librosa

 _ = librosa
 import matplotlib.pyplot as plt

 # TimeSeries -> Librosa (y, sr)
 y, sr = ts.to_librosa()
 print(f"--- Librosa Data ---\nSignal shape: {y.shape}, Sample rate: {sr}")

 # Plot using librosa style (matplotlib)
 plt.figure()
 plt.plot(y[:1000]) # Plot first 1000 samples
 plt.title("Librosa Audio Signal (Zoom)")
 plt.show()

 # Recover to TimeSeries
 from gwexpy.timeseries import TimeSeries

 ts_recovered = TimeSeries(y, dt=1.0 / sr)
 print("Recovered from Librosa:", ts_recovered)

 del y, sr
except ImportError:
 print("Librosa not installed.")

## 8. 医学・生体信号解析


### 8.1. MNE-Python <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **MNE**: MNE-Python は脳波 (EEG)・脳磁図 (MEG) データの解析ライブラリで、神経科学研究に広く使用されています。
> 📚 [MNE Documentation](https://mne.tools/)

脳電図 (EEG/MEG) 解析パッケージ MNE との連携。


In [ ]:
try:
 import mne

 _ = mne
 # TimeSeries -> MNE Raw
 raw = ts.to_mne()
 print("--- MNE Raw ---")
 print(raw)

 # Display info
 print(raw.info)

 # Recover to TimeSeries
 from gwexpy.timeseries import TimeSeries

 ts_recovered = TimeSeries.from_mne(raw, channel=ts.name or "ch0")
 print("Recovered from MNE:", ts_recovered)

 del raw, ts_recovered

except ImportError:
 print("MNE not installed.")

In [ ]:
try:
 import mne

 _ = mne
 raw = ts.to_mne()
 print("MNE Raw:", raw)

 del raw
except ImportError:
 print("MNE not installed.")

### 8.2. Elephant / quantities との連携 <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

`gwexpy` の `FrequencySeries` や `Spectrogram` は、`quantities.Quantity` オブジェクトと相互変換可能です。
これは Elephant や Neo との連携に役立ちます。

※ 事前に `pip install quantities` が必要です。


In [ ]:
try:
 import numpy as np
 import quantities as pq

 _ = pq
 from gwexpy.frequencyseries import FrequencySeries

 # FrequencySeries Create
 freqs = np.linspace(0, 100, 101)
 data_fs = np.random.random(101)
 fs = FrequencySeries(data_fs, frequencies=freqs, unit="V")

 # to_quantities
 q_obj = fs.to_quantities(units="mV")
 print("Quantities object:", q_obj)

 # from_quantities
 fs_new = FrequencySeries.from_quantities(q_obj, frequencies=freqs)
 print("Restored FrequencySeries unit:", fs_new.unit)

 del freqs, data_fs, fs, q_obj, fs_new

except ImportError:
 print("quantities library not found. Skipping example.")

### 8.3. Neo <span style="color: #2ecc71; font-weight: bold;">[GWExPy New]</span>

> **Neo**: Neo は電気生理学データ（神経科学）のためのデータ構造ライブラリで、様々なフォーマットへの入出力をサポートします。
> 📚 [Neo Documentation](https://neo.readthedocs.io/)

電気生理データの共通規格 Neo への変換をサポート。


In [ ]:
try:
 import neo

 _ = neo
 # TimeSeries -> Neo AnalogSignal
 # to_neo is available in gwexpy.interop
 # Note: TimeseriesMatrix is preferred for multi-channel Neo conversion,
 # but we can convert single TimeSeries by wrapping it.
 from gwexpy.interop import from_neo, to_neo

 _ = from_neo
 _ = to_neo
 # For single TimeSeries, we might need a Matrix wrapper or direct helper.
 # Assuming helper exists or using Matrix:
 from gwexpy.timeseries import TimeSeriesMatrix

 tm = TimeSeriesMatrix(
 ts.value[None, None, :], t0=ts.t0, dt=ts.dt, channel_names=[ts.name]
 )
 sig = tm.to_neo()

 print("--- Neo AnalogSignal ---")
 print(sig)

 # Display/Plot
 plt.figure()
 plt.plot(sig.times, sig)
 plt.title("Neo AnalogSignal Plot")
 plt.show()

 # Recover
 tm_recovered = TimeSeriesMatrix.from_neo(sig)
 ts_recovered = tm_recovered[0]
 print("Recovered from Neo:", ts_recovered)

 del tm, tm_recovered, sig, ts_recovered

except ImportError:
 print("neo not installed.")

## 9. まとめ

`gwexpy` は多種多様な 機械学習 向けライブラリとの相互運用を提供し、既存のエコシステムとのシームレスな統合を可能にします。
